In [30]:
#Interfaz
!pip install PySimpleGUI

In [ ]:
#Arbol - Librerias
!conda install -c etetoolkit ete3

!export PATH=~/anaconda_ete/bin:$PATH

!conda install -c bioconda iqtree


In [9]:
from ete3 import PhyloTree, faces, TreeStyle, NodeStyle
import subprocess

class Arbol:
    def __init__(self):
        test = subprocess.Popen(["iqtree","-s","./temp/fasta.fasta"], stdout=subprocess.PIPE)
        test.communicate()[0]
        self.alg = open("./temp/fasta.fasta", "r").read()
        self.newick = open("./temp/fasta.fasta.treefile", "r").read()
    
    def getPhyloTree(self):
        # Performs a tree reconciliation analysis 
        gene_tree_nw = self.newick
        genetree = PhyloTree(gene_tree_nw)
        genetree.link_to_alignment(self.alg)
        return genetree, TreeStyle()

    def armarArbol(self):
        # Visualize the reconciled tree
        t, ts = self.getPhyloTree()
        ts.scale = 20
        ts.show_leaf_name = True
        ts.show_branch_length = True
        ts.show_branch_support = True
        #ts.layout_fn = master_ly
        t.show(tree_style=ts)

arbol = Arbol()

In [5]:
#Mapa - libreria
!pip install geopy
!pip install folium
!pip install selenium

from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="TP FINAL Bioinformatica")

In [6]:
#Mapa - Modelo
import folium
from folium.plugins import MiniMap
from folium.plugins import MousePosition
import PySimpleGUI as sg
import io
from PIL import Image

class UbicacionNotFound(Exception):
    def __init__(self, nombre_ubicacion):
        super().__init__(f"La ubicación {nombre_ubicacion} no es válida")


class Mapa:
    def __init__(self):
        self.points = []
        self.my_map = folium.Map(
            zoom_start = 12,
            control_scale=True,
            world_copy_jump=False,
            no_wrap=False
        )
        
       
    def armar_mapa(self, nombres_de_ubicaciones):
        n = 1
        for nombre_ubicacion in nombres_de_ubicaciones:
            ubicacion = geolocator.geocode(nombre_ubicacion)
            if ubicacion == None:
                raise UbicacionNotFound(nombre_ubicacion)
            point = (ubicacion.latitude, ubicacion.longitude)
            self.points.append(point)
            folium.Marker(
                location = [point[0], point[1]],
                icon = folium.Icon(color='red', icon='cloud'),
                popup= f'Ubicacion: {nombre_ubicacion}',
                tooltip = n
            ).add_to(self.my_map)
            n = n + 1
        return self
    
    def render_mapa(self):
        MousePosition().add_to(self.my_map)
        folium.PolyLine(self.points).add_to(self.my_map)
        minimap = MiniMap()
        self.my_map.add_child(minimap)
        display(self.my_map)
            
mapa = Mapa()

In [10]:
import PySimpleGUI as sg

lista_de_nombre_de_ubicaciones = ["Londres", "Paris", "Amsterdam", "Las Vegas", "Tokyo", "Buenos Aires", "San Pablo", "Tijuana", "Medellin", "Texas", "Nueva Orleans"]

layoutSubirArchivo = [  [sg.Text('Filename')],
            [sg.Input(), sg.FileBrowse()], 
            [sg.OK(), sg.Cancel()]]

sg.theme('DarkAmber')   # Add a touch of color
# All the stuff inside your window.
layout = [  [sg.Text('MENU Pongamosle un lindo mensaje')],
            [sg.Button('Subir Fasta')],
            [sg.Text('O ingrese manualmente la secuencia.')],
            [sg.InputText()],
            [sg.Button('Cargar')],
            [sg.Button('Alinear Secuencias')],
            [sg.Button('Armar Arbol')],
            [sg.Button('Mostrar Mapa')],
         ]

window = sg.Window('Trabajo Final Bioinformatica', layout)
while True:
    event, values = window.read()
    if event == sg.WIN_CLOSED: 
        break
    elif event == 'Subir Fasta':
        event, values = sg.Window('Seleccione un archivo tipo Fasta', layoutSubirArchivo).read(close=True)
    elif event == 'Alinear Secuencias':
        alineacion.alinear()
    elif event == 'Armar Arbol':
        arbol.armarArbol()
    elif event == 'Mostrar Mapa':
        mapa.armar_mapa(lista_de_nombre_de_ubicaciones).render_mapa()

window.close()

